# 05 - MessagesPlaceholder 历史消息

## 用途
学习使用 LangChain MessagesPlaceholder 处理对话历史消息占位

## 学习目标
- 理解占位符作用机制
- 掌握动态填充历史消息
- 能与消息历史组件集成
- 控制历史消息长度以节省成本

## 代码块独立性说明
**注意**：每个代码块都是独立的，包含完整的导入和初始化，确保可以单独运行。

## 1. MessagesPlaceholder 基础使用

In [1]:
# MessagesPlaceholder 基础使用 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 初始化模型（使用轻量模型节省成本）
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=100,
)

# 创建带历史消息占位符的模板
chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是 GPT-4o 对话助手，根据历史上下文回答用户问题。"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{current_question}")
])

print("✅ MessagesPlaceholder 模板创建成功")
print(f"模板消息数量: {len(chat_template.messages)}")

# 准备历史消息（限制数量以控制成本）
history_messages = [
    HumanMessage(content="什么是 LangChain？"),
    AIMessage(content="LangChain 是一个用于构建 LLM 应用的框架。")
]

# 测试模板
current_question = "它有什么主要特点？"
formatted_messages = chat_template.format_messages(
    history=history_messages,
    current_question=current_question
)

print(f"\n📝 格式化后的消息:")
for i, msg in enumerate(formatted_messages):
    role = msg.__class__.__name__.replace('Message', '')
    print(f"  {i+1}. {role}: {msg.content}")

# 调用模型
response = llm.invoke(formatted_messages)
print(f"\n🤖 GPT-4o-mini 回复: {response.content}")

# 验证点：占位符能被历史消息正确填充
assert len(formatted_messages) == 4, "消息数量不正确"  # system + 2 history + user
assert "它有什么主要特点？" in formatted_messages[-1].content, "当前问题未正确填充"
assert len(response.content) > 0, "回复为空"
print("\n✅ 验证通过：MessagesPlaceholder 正确工作")

✅ MessagesPlaceholder 模板创建成功
模板消息数量: 3

📝 格式化后的消息:
  1. System: 你是 GPT-4o 对话助手，根据历史上下文回答用户问题。
  2. Human: 什么是 LangChain？
  3. AI: LangChain 是一个用于构建 LLM 应用的框架。
  4. Human: 它有什么主要特点？

🤖 GPT-4o-mini 回复: LangChain 的主要特点包括：

1. **模块化设计**：LangChain 提供了多个可组合的模块，允许开发者根据需求灵活构建应用。

2. **多种数据源支持**：它支持从不同的数据源获取信息，如 API、数据库和文档等。

3. **链式操作**：LangChain 允许将多个处理步骤串联在一起，形成数据处理和决策的链条。

4. **集成

✅ 验证通过：MessagesPlaceholder 正确工作


## 2. 动态历史消息管理

In [2]:
# 动态历史消息管理 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 初始化模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=80,
)

# 创建模板
template = ChatPromptTemplate.from_messages([
    ("system", "你是 GPT-4o 助手，根据对话历史提供简洁回答。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

print("✅ 动态历史消息模板创建成功")

# 模拟对话历史（限制长度控制成本）
conversation_history = [
    HumanMessage(content="你好，我想学习 Python。"),
    AIMessage(content="很好的选择！Python 是一门易学的编程语言。"),
    HumanMessage(content="Python 适合做什么？"),
    AIMessage(content="Python 适合数据分析、Web 开发、人工智能等领域。")
]

print(f"\n📚 当前历史消息数量: {len(conversation_history)}")

# 测试不同长度的历史消息
test_questions = [
    "推荐一个学习资源",
    "我该如何开始？"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n🧪 测试 {i}: {question}")
    
    # 使用部分历史消息（模拟动态管理）
    recent_history = conversation_history[-(i+1):]  # 动态调整历史长度
    
    formatted_messages = template.format_messages(
        chat_history=recent_history,
        input=question
    )
    
    response = llm.invoke(formatted_messages)
    
    print(f"   使用历史: {len(recent_history)} 条消息")
    print(f"   回复: {response.content}")

# 验证点：动态历史消息管理
assert len(conversation_history) == 4, "历史消息数量不正确"
print("\n✅ 验证通过：动态历史消息管理正常工作")

✅ 动态历史消息模板创建成功

📚 当前历史消息数量: 4

🧪 测试 1: 推荐一个学习资源
   使用历史: 2 条消息
   回复: 可以推荐《Python Crash Course》这本书，适合初学者快速入门。也可以访问 Coursera 或 edX 上的 Python 课程。

🧪 测试 2: 我该如何开始？
   使用历史: 3 条消息
   回复: 你可以通过以下步骤开始学习 Python：

1. **安装 Python**：访问 [Python 官网](https://www.python.org/) 下载并安装最新版本。
2. **选择学习资源**：可以使用在线课程（如 Coursera、Codecademy）、书籍（如《Python Crash Course》）或 YouTube 教程。
3. **实践编程**：通过编

✅ 验证通过：动态历史消息管理正常工作


## 3. 历史消息长度优化

In [3]:
# 历史消息长度优化 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("💰 历史消息长度优化测试:")
print("=" * 40)

# 初始化模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=60,
)

# 创建模板
template = ChatPromptTemplate.from_messages([
    ("system", "你是简洁的 GPT-4o 助手，根据上下文回答。"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])

# 生成不同长度的历史消息
def generate_history(length):
    """生成指定长度的历史消息"""
    history = []
    for i in range(length):
        if i % 2 == 0:
            history.append(HumanMessage(content=f"问题 {i//2 + 1}"))
        else:
            history.append(AIMessage(content=f"回答 {i//2 + 1}"))
    return history

# 测试不同历史长度的影响
history_lengths = [0, 2, 4, 6]  # 限制最大长度控制成本
test_question = "总结一下讨论的主要内容"

results = {}

for length in history_lengths:
    print(f"\n📏 测试历史长度: {length}")
    
    try:
        # 生成历史消息
        history = generate_history(length)
        
        # 格式化消息
        if length == 0:
            # 无历史消息的情况
            messages = [
                {"type": "system", "content": "你是简洁的 GPT-4o 助手。"},
                {"type": "user", "content": test_question}
            ]
            formatted_messages = [msg for msg in messages]  # 简化处理
        else:
            formatted_messages = template.format_messages(
                history=history,
                question=test_question
            )
        
        # 调用模型
        response = llm.invoke(formatted_messages)
        
        # 记录结果
        results[length] = {
            "response": response.content,
            "length": len(response.content),
            "success": True
        }
        
        print(f"   回复: {response.content}")
        print(f"   长度: {len(response.content)} 字符")
        print(f"   状态: ✅ 成功")
        
    except Exception as e:
        print(f"   状态: ❌ 失败 - {e}")
        results[length] = {
            "response": None,
            "length": 0,
            "success": False,
            "error": str(e)
        }

# 长度优化总结
print(f"\n📊 历史消息长度优化总结:")
successful_lengths = [length for length, result in results.items() if result["success"]]
print(f"✅ 成功测试的长度: {len(successful_lengths)}/{len(history_lengths)}")

if len(successful_lengths) >= 2:
    print("\n📈 不同历史长度回复对比:")
    for length in successful_lengths:
        result = results[length]
        print(f"   历史 {length} 条: {result['length']} 字符")
    
    print("\n💡 成本优化建议:")
    print("   - 限制历史消息数量（建议 2-4 条）")
    print("   - 使用消息摘要技术压缩长历史")
    print("   - 根据任务复杂度动态调整历史长度")
    print("   - 定期清理不相关的历史消息")
    print("✅ 验证通过：历史消息长度优化有效")
else:
    print("⚠️  警告：历史长度测试失败")

💰 历史消息长度优化测试:

📏 测试历史长度: 0
   回复: 当然，可以请您提供讨论的具体内容或主题吗？这样我能更好地为您总结。
   长度: 33 字符
   状态: ✅ 成功

📏 测试历史长度: 2
   回复: 请提供讨论的具体内容，我将为您总结主要要点。
   长度: 22 字符
   状态: ✅ 成功

📏 测试历史长度: 4
   回复: 请提供讨论的具体内容或主题，以便我为您总结主要内容。
   长度: 26 字符
   状态: ✅ 成功

📏 测试历史长度: 6
   回复: 请提供讨论的具体内容或主题，我将为您总结主要内容。
   长度: 25 字符
   状态: ✅ 成功

📊 历史消息长度优化总结:
✅ 成功测试的长度: 4/4

📈 不同历史长度回复对比:
   历史 0 条: 33 字符
   历史 2 条: 22 字符
   历史 4 条: 26 字符
   历史 6 条: 25 字符

💡 成本优化建议:
   - 限制历史消息数量（建议 2-4 条）
   - 使用消息摘要技术压缩长历史
   - 根据任务复杂度动态调整历史长度
   - 定期清理不相关的历史消息
✅ 验证通过：历史消息长度优化有效


## 4. 链式调用集成

In [4]:
# 链式调用集成 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 初始化模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=80,
)

# 创建带历史消息的模板
chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是 GPT-4o 助手，根据对话历史提供连贯回答。"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{input}")
])

# 创建 LCEL 链：chat_template | llm | parser
chain = chat_template | llm | StrOutputParser()

print("✅ MessagesPlaceholder 链式调用创建成功")
print("🔗 链结构: ChatPromptTemplate → ChatOpenAI (GPT-4o-mini) → StrOutputParser")

# 准备历史消息
history = [
    HumanMessage(content="什么是机器学习？"),
    AIMessage(content="机器学习是让计算机从数据中学习的技术。")
]

# 测试链式调用
result = chain.invoke({
    "history": history,
    "input": "能举个例子吗？"
})

print(f"\n🤖 链式调用结果: {result}")
print(f"📊 结果类型: {type(result)}")
print(f"📊 结果长度: {len(result)} 字符")

# 验证点：链式调用返回字符串格式结果
assert isinstance(result, str), "结果不是字符串类型"
assert len(result) > 0, "结果为空"
print("\n✅ 验证通过：MessagesPlaceholder 链式调用成功")

✅ MessagesPlaceholder 链式调用创建成功
🔗 链结构: ChatPromptTemplate → ChatOpenAI (GPT-4o-mini) → StrOutputParser

🤖 链式调用结果: 当然可以！一个常见的机器学习例子是垃圾邮件过滤。 

在这个应用中，机器学习算法会分析大量的电子邮件，学习哪些特征通常出现在垃圾邮件中（比如特定的关键词、发件人地址或邮件格式）。通过对这些特征的学习，算法能够根据新的邮件内容判断它们是否可能是垃圾邮件。当
📊 结果类型: <class 'str'>
📊 结果长度: 127 字符

✅ 验证通过：MessagesPlaceholder 链式调用成功


## 5. 学习总结与最佳实践

In [5]:
# 学习总结与最佳实践 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("📋 MessagesPlaceholder 学习总结:")
print("=" * 40)

# 验证点检查
verification_points = [
    "✅ 基础占位符：history 变量动态填充",
    "✅ 动态历史管理：可变长度历史消息",
    "✅ 长度优化：成本控制策略",
    "✅ 链式调用：LCEL 语法集成",
]

for point in verification_points:
    print(point)

print(f"\n🎯 核心技能掌握情况: {len(verification_points)}/4 项")

print("\n💡 MessagesPlaceholder 最佳实践:")
print("1. 历史长度：限制在 2-6 条消息控制成本")
print("2. 消息格式：使用 HumanMessage/AIMessage 标准格式")
print("3. 动态管理：根据上下文相关性调整历史")
print("4. 成本控制：优先使用 gpt-4o-mini 模型")
print("5. 链式调用：结合 LCEL 提高开发效率")

print("\n🚀 下一步学习建议:")
print("1. 深入学习 Structured Output 结构化输出")
print("2. 掌握 ConversationBufferMemory 记忆管理")
print("3. 学习 ConversationSummaryMemory 摘要记忆")
print("4. 探索 RunnableWithMessageHistory 历史集成")
print("5. 实践多轮对话状态管理")

# 最终验证：确保 MessagesPlaceholder 基础功能可用
try:
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.7,
        max_tokens=40,
    )
    
    template = ChatPromptTemplate.from_messages([
        ("system", "你是简洁的 GPT-4o 助手。"),
        MessagesPlaceholder(variable_name="history"),
        ("user", "用一句话说明你学会了 {skill}")
    ])
    
    history = [HumanMessage(content="你好")]
    response = llm.invoke(template.format_messages(
        history=history,
        skill="MessagesPlaceholder"
    ))
    
    print(f"\n🎉 最终验证成功: {response.content}")
    print("\n✅ MessagesPlaceholder 学习完成！")
    
except Exception as e:
    print(f"\n❌ 最终验证失败: {e}")

📋 MessagesPlaceholder 学习总结:
✅ 基础占位符：history 变量动态填充
✅ 动态历史管理：可变长度历史消息
✅ 长度优化：成本控制策略
✅ 链式调用：LCEL 语法集成

🎯 核心技能掌握情况: 4/4 项

💡 MessagesPlaceholder 最佳实践:
1. 历史长度：限制在 2-6 条消息控制成本
2. 消息格式：使用 HumanMessage/AIMessage 标准格式
3. 动态管理：根据上下文相关性调整历史
4. 成本控制：优先使用 gpt-4o-mini 模型
5. 链式调用：结合 LCEL 提高开发效率

🚀 下一步学习建议:
1. 深入学习 Structured Output 结构化输出
2. 掌握 ConversationBufferMemory 记忆管理
3. 学习 ConversationSummaryMemory 摘要记忆
4. 探索 RunnableWithMessageHistory 历史集成
5. 实践多轮对话状态管理

🎉 最终验证成功: 我学会了使用 MessagesPlaceholder 来处理和替换消息中的占位符。

✅ MessagesPlaceholder 学习完成！
